In [ ]:
from bs4 import BeautifulSoup
import requests
import re 

html = requests.get(
    "https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=chip-visao-neuromorfica-promete-visao-instantanea-carros-robos&id=010150250515").content

soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# Seção
secao = soup.find('div', class_='secao')
secao = secao.get_text(strip=True) if secao else None

# Título
titulo = soup.find('h1')
titulo = titulo.get_text(strip=True) if titulo else None

# Fonte e data
autor_data = soup.find('p', class_='suave')
if autor_data:
    autor_data_text = autor_data.get_text(strip=True)
    
    # Expressão regular para separar fonte e data com hífen colado ou com espaços
    match = re.match(r'^(.*?)[\s\-–]+(\d{2}/\d{2}/\d{4})$', autor_data_text)
    if match:
        fonte, data = match.group(1).strip(), match.group(2)
    else:
        fonte, data = autor_data_text, None
else:
    fonte, data = None, None

# Legenda
legenda_div = soup.find('div', class_='legenda')
if legenda_div:
    legenda_texto = legenda_div.get_text(separator=' ', strip=True)
    legenda = legenda_texto.split('[Imagem:')[0].strip()
else:
    legenda = None

# Exibe os resultados
print("Seção:", secao)
print("Título:", titulo)
print("Fonte:", fonte)
print("Data:", data)
print("Legenda:", legenda)

In [ ]:

paragrafos = soup.find_all('p')

texto_noticia = []
for p in paragrafos:
    txt = p.get_text(strip=True)
    if (
        not txt or
        "redação do site inovação tecnológica" in txt.lower() or
        "mais tópicos" in txt.lower()
    ):
        continue
    texto_noticia.append(txt)

texto_final = '\n\n'.join(texto_noticia)
print(texto_final)

In [ ]:
div_biblio = soup.find('div', class_='biblio')

# Separa o texto por linhas removendo espaços extras
linhas = [linha.strip() for linha in div_biblio.get_text(separator='\n').split('\n') if linha.strip()]

# Inicializa variáveis
autores = None
revista = None

# Percorre as linhas para encontrar autores e revista
for linha in linhas:
    if linha.startswith('Autores:'):
        autores = linha.replace('Autores:', '').strip()
    elif linha.startswith('Revista:'):
        revista = linha.replace('Revista:', '').strip()

print('Autores:', autores)
print('Revista:', revista)

In [19]:
from bs4 import BeautifulSoup
import requests
import re 
import json
import unicodedata

def normalizar_texto(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    texto = re.sub(r'\s+', '-', texto)
    texto = re.sub(r'[^\w\-]', '', texto)  # Remove caracteres que não são letra, número ou hífen
    return texto




html = requests.get(
    "https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=chip-visao-neuromorfica-promete-visao-instantanea-carros-robos&id=010150250515").content

soup = BeautifulSoup(html, 'html.parser')

# Seção
secao = soup.find('div', class_='secao')
secao = secao.get_text(strip=True) if secao else None

# Título
titulo = soup.find('h1')
titulo = titulo.get_text(strip=True) if titulo else None

# Fonte e data
autor_data = soup.find('p', class_='suave')
if autor_data:
    autor_data_text = autor_data.get_text(strip=True)
    
    # Expressão regular para separar fonte e data com hífen colado ou com espaços
    match = re.match(r'^(.*?)[\s\-–]+(\d{2}/\d{2}/\d{4})$', autor_data_text)
    if match:
        fonte, data = match.group(1).strip(), match.group(2)
    else:
        fonte, data = autor_data_text, None
else:
    fonte, data = None, None

# Legenda
legenda_div = soup.find('div', class_='legenda')
if legenda_div:
    legenda_texto = legenda_div.get_text(separator=' ', strip=True)
    legenda = legenda_texto.split('[Imagem:')[0].strip()
else:
    legenda = None



paragrafos = soup.find_all('p')

texto_noticia = []
for p in paragrafos:
    txt = p.get_text(strip=True)
    if (
        not txt or
        "redação do site inovação tecnológica" in txt.lower() or
        "mais tópicos" in txt.lower()
    ):
        continue
    texto_noticia.append(txt)

texto_final = '\n\n'.join(texto_noticia)



div_biblio = soup.find('div', class_='biblio')

# Separa o texto por linhas removendo espaços extras
linhas = [linha.strip() for linha in div_biblio.get_text(separator='\n').split('\n') if linha.strip()]

# Inicializa variáveis
autores = None
revista = None

# Percorre as linhas para encontrar autores e revista
for linha in linhas:
    if linha.startswith('Autores:'):
        autores = linha.replace('Autores:', '').strip()
    elif linha.startswith('Revista:'):
        revista = linha.replace('Revista:', '').strip()


arquivo = {
        "Origem": "Inovação Tecnológica",
        "Seção": secao,
        "Título": titulo,
        "Fonte": fonte,
        "Data": data,
        "Legenda": legenda,
        "Texto": texto_final,
        "Autor": autores,
        "Publicação": revista
    }

nome_arquivo = normalizar_texto(titulo) if titulo else 'arquivo'

caminho_pasta = "'data/noticias/inovacao-tecnologica/informatica'"

    # Salva arquivo JSON
caminho_arquivo = f"data/noticias/inovacao-tecnologica/informatica/{nome_arquivo}.json"
with open(caminho_arquivo, 'w', encoding='utf-8') as f:
    json.dump(arquivo, f, ensure_ascii=False, indent=4)


In [ ]:
import re
import json
import unicodedata
from bs4 import BeautifulSoup

def salvar_json_de_soup(soup, caminho_pasta):
    # Função para normalizar string (remover acentos e trocar espaços por hífen)
    def normalizar_texto(texto):
        texto = texto.lower()
        texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
        texto = re.sub(r'\s+', '-', texto)
        texto = re.sub(r'[^\w\-]', '', texto)  # Remove caracteres que não são letra, número ou hífen
        return texto

    # Seção
    secao = soup.find('div', class_='secao')
    secao = secao.get_text(strip=True) if secao else None

    # Título
    titulo = soup.find('h1')
    titulo = titulo.get_text(strip=True) if titulo else None

    # Fonte e data
    autor_data = soup.find('p', class_='suave')
    if autor_data:
        autor_data_text = autor_data.get_text(strip=True)
        # Expressão regular para separar fonte e data com hífen colado ou com espaços
        match = re.match(r'^(.*?)[\s\-–]+(\d{2}/\d{2}/\d{4})$', autor_data_text)
        if match:
            fonte, data = match.group(1).strip(), match.group(2)
        else:
            fonte, data = autor_data_text, None
    else:
        fonte, data = None, None

    # Legenda
    legenda_div = soup.find('div', class_='legenda')
    if legenda_div:
        legenda_texto = legenda_div.get_text(separator=' ', strip=True)
        legenda = legenda_texto.split('[Imagem:')[0].strip()
    else:
        legenda = None

    # Texto da notícia (filtrando parágrafos indesejados)
    paragrafos = soup.find_all('p')
    texto_noticia = []
    for p in paragrafos:
        txt = p.get_text(strip=True)
        if (
            not txt or
            "redação do site inovação tecnológica" in txt.lower() or
            "mais tópicos" in txt.lower()
        ):
            continue
        texto_noticia.append(txt)
    texto_final = '\n\n'.join(texto_noticia)

    # Autores e revista na bibliografia
    div_biblio = soup.find('div', class_='biblio')
    autores = None
    revista = None
    if div_biblio:
        linhas = [linha.strip() for linha in div_biblio.get_text(separator='\n').split('\n') if linha.strip()]
        for linha in linhas:
            if linha.startswith('Autores:'):
                autores = linha.replace('Autores:', '').strip()
            elif linha.startswith('Revista:'):
                revista = linha.replace('Revista:', '').strip()

    # Monta o dicionário final
    arquivo = {
        "Origem": "Inovação Tecnológica",
        "Seção": secao,
        "Título": titulo,
        "Fonte": fonte,
        "Data": data,
        "Legenda": legenda,
        "Texto": texto_final,
        "Autor": autores,
        "Publicação": revista
    }

    # Normaliza o título para criar o nome do arquivo
    nome_arquivo = normalizar_texto(titulo) if titulo else 'arquivo'

    # Salva arquivo JSON
    caminho_arquivo = f"{caminho_pasta.rstrip('/')}/{nome_arquivo}.json"
    with open(caminho_arquivo, 'w', encoding='utf-8') as f:
        json.dump(arquivo, f, ensure_ascii=False, indent=4)

    return arquivo, caminho_arquivo


In [ ]:
from bs4 import BeautifulSoup

# html = requests.get(
#     "https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=chip-visao-neuromorfica-promete-visao-instantanea-carros-robos&id=010150250515").content


# exemplo: carregue seu HTML na variável html
soup = BeautifulSoup("https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=chip-visao-neuromorfica-promete-visao-instantanea-carros-robos&id=010150250515", 'html.parser')

dados, caminho = salvar_json_de_soup(soup, 'data/noticias/inovacao-tecnologica/informatica')  # './dados' é a pasta onde salvará o arquivo

print(f"Arquivo salvo em: {caminho}")
print(dados)


In [ ]:
print(soup.prettify()[:1000])  # Imprime os primeiros 1000 caracteres do HTML parseado
https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=chip-visao-neuromorfica-promete-visao-instantanea-carros-robos&amp;id=010150250515

In [ ]:
print(soup.find('div', class_='secao'))
print(soup.find('h1'))
print(soup.find('p', class_='suave'))
print(soup.find('div', class_='legenda'))
print(soup.find('div', class_='biblio'))


In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.inovacaotecnologica.com.br/noticias/noticia.php?artigo=chip-visao-neuromorfica-promete-visao-instantanea-carros-robos&id=010150250515'

resp = requests.get(url)
print(resp.status_code)  # deve ser 200
print(resp.url)          # veja se redirecionou para outra página
print(resp.encoding)

soup = BeautifulSoup(resp.text, 'html.parser')

print(soup.prettify()[:1000])
